In [2]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

In [5]:
# 학습이 완료된 모델을 저장할 파일 이름
best_model_path = 'best_model.dat'
# 교차검증 횟수
cv_count = 10
# 교차 검증
kfold = KFold(n_splits=cv_count, shuffle=True, random_state=1)
# 평가 결과를 담을 리스트
# 필요하다면 다른 것도 만들어주세요
f1_score_list = []
# 학습 모델 이름
model_name_list = []

In [7]:
# 데이터를 읽어온다.
r05 = pd.read_parquet('open/r05.parquet')

In [ ]:
# 표준화를 위해 결과데이터를 제외한다 (Segment와 ID 컬럼 제외)
X_raw = r05.drop(['Segment', 'ID'], axis=1) # 원본 X (스케일링 전)

# 표준화 스케일러 학습 (fit)
scaler1 = StandardScaler()
scaler1.fit(X_raw) # r05.drop(['Segment', 'ID'], axis=1)와 동일한 데이터로 fit

# 입력(특성)과 결과(타겟)로 나눈다.
# 이 단계에서 X는 원본 r05에서 Segment와 ID를 제외한 것
X = r05.drop(['Segment', 'ID'], axis = 1)
y = r05['Segment'] # 타겟 변수

# 표준화 적용 (transform)
X_scaled = scaler1.transform(X) # X는 NumPy 배열이 됩니다.
# X_scaled의 컬럼 이름을 유지하기 위해 DataFrame으로 변환 (권장)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

# 여기까지 현재 진행 상황
print("\n--- 표준화된 X_scaled_df와 y 확인 ---")
print("X_scaled_df.head():")
print(X_scaled_df.head())
print("\ny.head():")
print(y.head())
print(f"X_scaled_df shape: {X_scaled_df.shape}")
print(f"y shape: {y.shape}")


# --- 이 다음 단계 ---

# 1. 훈련(Train) 세트와 테스트(Test) 세트로 분할
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder # y가 문자열인 경우 대비

# y (타겟 변수)가 문자열 범주형인 경우 Label Encoding이 필요합니다.
# 'Segment' 컬럼이 문자열('A', 'B', 'C' 등)이라면, 모델 학습 전에 숫자로 변환해야 합니다.
# 이미 숫자로 인코딩되어 있다면 이 부분은 건너뛰세요.
if y.dtype == 'object' or pd.api.types.is_categorical_dtype(y):
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    print(f"\n--- 타겟 변수 Label Encoding 완료 ({le.classes_}) ---")
    print(f"원본 y head: {y.head()}")
    print(f"인코딩된 y head: {pd.Series(y_encoded).head()}")
else:
    y_encoded = y # 이미 숫자형인 경우 그대로 사용
    print(f"\n--- 타겟 변수는 이미 숫자형입니다. Label Encoding 건너뛰었습니다. ---")


# X_scaled_df (표준화된 특성 데이터프레임)와 y_encoded (인코딩된 타겟)를 분할
# test_size=0.2: 전체 데이터의 20%를 테스트 세트로 사용 (80%는 훈련 세트)
# random_state=42: 재현성을 위한 난수 시드 고정
# stratify=y_encoded: y의 클래스 비율을 훈련/테스트 세트에 동일하게 유지 (클래스 불균형 데이터에 중요)
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

print("\n--- 데이터 분할 결과 ---")
print(f"훈련 데이터 (X_train) shape: {X_train.shape}")
print(f"테스트 데이터 (X_test) shape: {X_test.shape}")
print(f"훈련 타겟 (y_train) shape: {y_train.shape}")
print(f"테스트 타겟 (y_test) shape: {y_test.shape}")


# 이제 X_train, y_train을 가지고 모델 학습을 시작할 수 있습니다.
# 그리고 X_test, y_test를 가지고 모델 성능을 평가할 수 있습니다.

# 다음으로 진행할 내용은 모델 학습 및 평가입니다.
# 예시: LogisticRegression 학습 및 평가
from sklearn.linear_model import LogisticRegression # <-- LogisticRegression 임포트
from sklearn.metrics import accuracy_score, classification_report

print("\n--- LogisticRegression 모델 학습 시작 ---")
# LogisticRegression 초기화
# solver: 대규모 데이터셋에 적합한 'saga' 또는 'lbfgs'를 추천합니다.
#         'saga'는 L1/L2 규제 모두 지원하고 대규모 데이터에 잘 작동합니다.
# multi_class: 다중 클래스 분류의 경우 'multinomial' (softmax 회귀) 또는 'ovr' (one-vs-rest)
#              'multinomial'은 실제 다중 클래스 문제를 한 번에 풀고, 'ovr'은 각 클래스별 이진 분류기를 만듭니다.
# max_iter: 경사 하강법 반복 횟수. 데이터가 크므로 충분히 크게 설정해야 수렴할 수 있습니다.
# n_jobs=-1: 모든 CPU 코어 사용 (solver='saga'에서 지원)
model = LogisticRegression(random_state=42, n_jobs=-1,
                           solver='saga', multi_class='multinomial', max_iter=500) # max_iter 조정 필요할 수 있음
model.fit(X_train, y_train)
print("모델 학습 완료!")

print("\n--- 테스트 데이터 예측 ---")
y_pred = model.predict(X_test)
print("예측 완료.")

print("\n--- 모델 성능 평가 ---")
accuracy = accuracy_score(y_test, y_pred)
print(f"모델 정확도 (Accuracy): {accuracy:.4f}")

# 분류 리포트 (Precision, Recall, F1-score for each class)
# LabelEncoder를 사용했으므로, le.classes_를 사용하여 원본 클래스 이름을 출력합니다.
print("\n--- 분류 리포트 ---")
print(classification_report(y_test, y_pred, target_names=le.classes_ if 'le' in locals() else None))

# 예측 결과 직접 확인하는 부분 (이전 대화에서 추가한 부분)
print("\n--- 예측 값과 실제 값 비교 (테스트 세트 상위 10개) ---")
if 'le' in locals(): # LabelEncoder 객체가 정의되어 있다면 (y가 문자열이었던 경우)
    actual_segments = le.inverse_transform(y_test[:10]) # 실제값 상위 10개
    predicted_segments = le.inverse_transform(y_pred[:10]) # 예측값 상위 10개

    comparison_df = pd.DataFrame({
        'Actual Segment': actual_segments,
        'Predicted Segment': predicted_segments
    })
    print(comparison_df)
else: # y가 처음부터 숫자형이었던 경우
    comparison_df = pd.DataFrame({
        'Actual Segment': y_test[:10],
        'Predicted Segment': y_pred[:10]
    })
    print(comparison_df)

print("\n--- 예측된 'Segment' 값들의 분포 ---")
if 'le' in locals():
    predicted_counts = pd.Series(le.inverse_transform(y_pred)).value_counts().sort_index()
    print(predicted_counts)
else:
    predicted_counts = pd.Series(y_pred).value_counts().sort_index()
    print(predicted_counts)


# 모델 및 스케일러 저장 (선택 사항)
import pickle
with open('scaler_r05.pkl', 'wb') as f:
    pickle.dump(scaler1, f)
with open('model_r05_lr.pkl', 'wb') as f: # 파일명에 lr 추가하여 구분
    pickle.dump(model, f)
print("\n스케일러와 LogisticRegression 모델이 각각 'scaler_r05.pkl', 'model_r05_lr.pkl'로 저장되었습니다.")


--- 표준화된 X_scaled_df와 y 확인 ---
X_scaled_df.head():
   이용금액_R3M_신용체크  이용금액_R3M_신용  _1순위카드이용금액     기준년월  정상청구원금_B0M  정상청구원금_B2M  \
0      -0.720137    -0.689131   -0.520925 -1.46385    1.452885    1.656513   
1      -0.720137    -0.689131   -0.520925 -1.46385    1.452885    1.656513   
2      -0.720137    -0.689131   -0.520925 -1.46385    1.452885    1.656513   
3      -0.720137    -0.689131   -0.520925 -1.46385    1.452885    1.656513   
4      -0.720137    -0.689131   -0.520925 -1.46385    1.452885    1.656513   

   이용금액_일시불_R12M  이용금액_일시불_B0M  이용금액_일시불_R6M  이용금액_일시불_R3M  이용금액_오프라인_B0M  \
0      -0.355652     -0.345799     -0.473516     -0.559988        0.11058   
1      -0.355652     -0.345799     -0.473516     -0.559988        0.11058   
2      -0.355652     -0.345799     -0.473516     -0.559988        0.11058   
3      -0.355652     -0.345799     -0.473516     -0.559988        0.11058   
4      -0.355652     -0.345799     -0.473516     -0.559988        0.11058   

   정상입금원금_B5M  정